In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
# competition_url = "https://99designs.hk/logo-design/contests/logo-design-wanted-pure-water-technology-63431"
# competition_url = "https://99designs.hk/logo-design/contests/logo-design-appointmentpost-63470"
# competition_url = "https://99designs.hk/logo-design/contests/eye-pleasing-logo-weight-wellness-reset-program-transform-1193088"

In [3]:
PATH = "C:\\Users\\support\\Downloads\\chromedriver_win32\\chromedriver.exe" 

In [4]:
def scrape(competition_url):

    def make_soup(competition_url, page_number, active=True):

        kv = {'user-agent': 'Mozilla/5.0'}
        if active:
            url = competition_url + "/entries?filter=active&page=" + str(page_number)
        else: 
            url = competition_url + "/entries?filter=non_active&page=" + str(page_number)

        r = requests.get(url, headers=kv, timeout=30)
        r.raise_for_status()
        r.status_code
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, 'html.parser')
        return soup



    def get_time(soup):
        ## get time
        # text = str(soup.find(name='div', attrs='entry-pane__results').contents)
        text = str(soup.find(name='div', attrs='entry-pane__results'))
        time = re.findall('"timeCreatedString":".{20,30}"', text)
        if time:
            time = list(map(lambda x: x.replace('"timeCreatedString":"', '').replace('","', ""), time))
            return time

        time = re.findall('timeCreatedString\&quot\;\:&quot;.{20,30}\&quot\;', text)
        if time:
            time = list(map(lambda x: x[30: -6], time))
            return time

    participants_user_ids = []
    participants_entry_ids = []
    participants_entry_image_urls = []
    participants_submission_time = []

    def get_participant_and_entry_info(soup):

        entry_matrix = soup.findAll(name='div', attrs='entry-matrix__item matrix__item')
        for entry in entry_matrix:
            result = entry.find(name='div', attrs='entry entry--linked entry--zoom-linked')
            try:
                participants_user_ids.append(result["data-user-id"])
                participants_entry_ids.append(result["id"])
                participants_entry_image_urls.append(result.find(name='a')["href"])
            except:
                pass

    def get_ratings(competition_url, page_number, active=True):
        if active:
            url = competition_url + "/entries?filter=active&page=" + str(page_number)
        else: 
            url = competition_url + "/entries?filter=non_active&page=" + str(page_number)
        
        driver = webdriver.Chrome(PATH)
        driver.get(url)
        entry_information = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[3]/div/div/div[2]/div[5]/div[1]/div/div').get_attribute('data-design-collection')
        ratings = re.findall("\"rating\":\d|\"rating\":false", entry_information)
        ratings = list(map(lambda x: x.replace("\"rating\":", ""), ratings))
        driver.close()
        return ratings

    # non deleted entries
    ratings = []
    page_number = 1


    while True:
        # print(page_number)
        if page_number == 1:
            rating = get_ratings(competition_url, page_number)
            ratings += rating
            if len(rating) < 36:
                break
        else:

            rating = get_ratings(competition_url, page_number)
            ratings += rating
            if len(rating) < 36:
                break


        page_number += 1



    while True:
        if page_number == 1:
            soup = make_soup(competition_url, page_number)
            get_participant_and_entry_info(soup)
            time = get_time(soup)
            participants_submission_time += time
            if len(time) < 36:
                break
        else:
            soup = make_soup(competition_url, page_number)
            get_participant_and_entry_info(soup)
            time = get_time(soup)
            participants_submission_time += time
            if len(time) < 36:
                break

        # print(participants_user_ids)
        # print(participants_entry_ids)
        # print(participants_entry_image_urls)

        page_number += 1


    entries = []
    for entry_id, participant_id, time, url, rating in zip(participants_entry_ids, 
                                                    participants_user_ids, 
                                                    participants_submission_time, 
                                                    participants_entry_image_urls,
                                                    ratings):
        entries.append({'entry_id': entry_id, 'participant_id': participant_id, 'time': time, 'url':url, 'rating': rating})


    page_number = 1

    def get_time_and_status_for_deleted_entry(entry_id):
        entry_id = entry_id.split('-')[-1]
        entry_url = competition_url + '/entries/' + entry_id
        
        ## make soup for entry page
        kv = {'user-agent': 'Mozilla/5.0'}
        r = requests.get(entry_url, headers=kv, timeout=30)
        r.raise_for_status()
        r.status_code
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, 'html.parser')

        ## 
        script = str(soup.find('script', id="standalone-design-details-app-data"))
        time = re.search('"timeCreatedString":".{20,30}"', script).group(0)
        time = time.replace('"timeCreatedString":"', '').replace('","', "")

        status = 'deleted'
        if re.search('"status":"withdrawn"', script):
            status = 'withdrawn'
        elif re.search('"status":"eliminated"', script):
            status = 'declined'
        

        return time, status
        


    # In[33]:


    def get_participant_and_entry_info_deleted_page(soup):

        user_ids = []
        entry_ids = []
        status = []
        submission_time = []
        deleted_count = 0
        withdrawn_count = 0
        declined_count = 0

        entry_matrix = soup.findAll(name='div', attrs='entry-matrix__item matrix__item')
        
        for entry in entry_matrix:
            result = entry.find(name='div', attrs='entry')
        # print(result)
            try:
                user_ids.append(result["data-user-id"])
                entry_ids.append(result["id"])
                time, s = get_time_and_status_for_deleted_entry(result["id"])
                submission_time.append(time)
                status.append(s)

                if s == 'deleted':
                    deleted_count += 1
                elif s == 'withdrawn':
                    withdrawn_count += 1  
                elif s == 'declined':
                    declined_count += 1
                    

            except:
                pass

        return user_ids, entry_ids, status, submission_time, deleted_count, withdrawn_count, declined_count


    # In[34]:


    deleted_participants_user_ids = []
    deleted_participants_entry_ids = []
    status_all = []
    submission_time_all = []
    total_deleted_count = 0
    total_withdrawn_count = 0
    total_declined_count = 0

    page_number = 1

    while True:
        # print(page_number)
        if page_number == 1:
            soup = make_soup(competition_url, page_number, active=False)
            user_ids, entry_ids, status, submission_time, deleted_count, withdrawn_count, declined_count = get_participant_and_entry_info_deleted_page(soup)
            deleted_participants_user_ids += user_ids
            deleted_participants_entry_ids += entry_ids
            status_all += status
            submission_time_all += submission_time
            total_deleted_count += deleted_count
            total_withdrawn_count += withdrawn_count
            total_declined_count += declined_count
            if len(user_ids) < 36:
                break
        else:
            soup = make_soup(competition_url, page_number, active=False)
            user_ids, entry_ids, status, submission_time, deleted_count, withdrawn_count, declined_count = get_participant_and_entry_info_deleted_page(soup)
            deleted_participants_user_ids += user_ids
            deleted_participants_entry_ids += entry_ids
            status_all += status
            submission_time_all += submission_time
            total_deleted_count += deleted_count
            total_withdrawn_count += withdrawn_count
            total_declined_count += declined_count
            if len(user_ids) < 36:
                break

        page_number += 1


    # deleted entries
    page_number = 1
    deleted_ratings = []
    while True:
        # print(page_number)
        if page_number == 1:
            rating = get_ratings(competition_url, page_number, False)
            deleted_ratings += rating
            if len(rating) < 36:
                break
        else:

            rating = get_ratings(competition_url, page_number, False)
            deleted_ratings += rating
            if len(rating) < 36:
                break


        page_number += 1


    deleted_entries = []
    for entry_id, participant_id, status, submission_time, rating in zip(deleted_participants_entry_ids, 
                                        deleted_participants_user_ids, status_all, submission_time_all, deleted_ratings):
        deleted_entries.append({'entry_id': entry_id, 'participant_id': participant_id, 'status': status, 'time': submission_time, 'rating': rating})

    df_competition_description = pd.DataFrame({'url': competition_url, 'rating_entries': [entries],
                                                    'rating_deleted_entries': [deleted_entries]})
    
    return df_competition_description

In [5]:
urls = pd.read_csv('url_2022-02-01-to-2022-10-03.csv')
urls = urls['href'].values[:3000]

results = pd.DataFrame({})

for url in tqdm(urls):
    try:
        df = scrape(url)
        results = pd.concat([results, df])
    except:
        pass

  2%|▏         | 59/3000 [4:03:06<284:53:28, 348.73s/it]

In [ ]:
results.to_csv('contest_data_ratings_part1.csv', index=False)